In [2]:
import os

# Directories containing the uncropped images
directories = ["./raw_images"]
file_exts = [[os.path.splitext(file)[1] for file in os.listdir(dir)] for dir in directories]
print(list(set([ext for exts in file_exts for ext in exts])))


['', '.jpg', '.JPG', '.JPEG', '.PNG', '.jpeg', '.png']


In [3]:
from PIL import Image
import pillow_heif

pillow_heif.register_heif_opener()

for dir in directories:
    files = os.listdir(dir)
    for file in files:
        # Delete videos
        if file.lower().endswith(".mov") or file.lower().endswith(".mp4"):
            os.remove(os.path.join(dir, file))
            print(f"Deleted file: {os.path.join(dir, file)}")

        # Convert HEIC images to JPG
        if file.lower().endswith(".heic"):
            heic_image = Image.open(os.path.join(dir, file))
            jpg_file = os.path.splitext(file)[0] + ".jpg"
            heic_image.save(os.path.join(dir, jpg_file), format="JPEG")
            os.remove(os.path.join(dir, file))
            print(f"HEIC Image {os.path.join(dir, file)} has been converted to {os.path.join(dir, jpg_file)}")

In [4]:
# Just to make sure the extensions are correct now
converted_file_exts = [[os.path.splitext(file)[1] for file in os.listdir(dir)] for dir in directories]
print(list(set([ext for exts in converted_file_exts for ext in exts])))

['', '.jpg', '.JPG', '.JPEG', '.PNG', '.jpeg', '.png']


In [6]:
from mtcnn.mtcnn import MTCNN
import numpy as np

reached_last_image = True
last_face_count = 4000
last_cropped_image = "./raw_benny/IMG_1960.JPG"

face_dir = "./faces"
if not os.path.exists(face_dir):
    os.makedirs(face_dir)

for dir in directories:
    for file in os.listdir(dir):

        # For continuing from last image cropped
        if not reached_last_image:
            if os.path.join(dir, file) == last_cropped_image:
                reached_last_image = True
            continue
        if not (file.lower().endswith(".jpg") or file.lower().endswith(".png") or file.lower().endswith(".jpeg")):
            continue

        image = Image.open(os.path.join(dir, file))
        if image.mode != "RGB":
            image = image.convert("RGB")
        pixel_arr = np.array(image)

        detector = MTCNN()
        faces = detector.detect_faces(pixel_arr)
        for face in faces:
            x, y, width, height = face['box']
            cropped_face = image.crop((x, y, x + width, y + height))
            face_url = os.path.join(face_dir, f"face{str(last_face_count)}.jpg")
            cropped_face.save(face_url)
            print(f"Cropped face {str(last_face_count)} from {os.path.join(dir, file)}")
            last_face_count += 1

8/8 [==============================] - 0s 7ms/step
Cropped face 4000 from ./raw_images/IMG_7974.JPG
Cropped face 4001 from ./raw_images/IMG_7974.JPG
Cropped face 4002 from ./raw_images/IMG_7974.JPG
Cropped face 4003 from ./raw_images/IMG_7974.JPG
Cropped face 4004 from ./raw_images/IMG_7974.JPG
Cropped face 4005 from ./raw_images/IMG_7974.JPG
Cropped face 4006 from ./raw_images/IMG_7974.JPG
Cropped face 4007 from ./raw_images/IMG_7974.JPG
Cropped face 4008 from ./raw_images/IMG_7974.JPG
Cropped face 4009 from ./raw_images/IMG_7974.JPG
Cropped face 4010 from ./raw_images/IMG_7974.JPG
Cropped face 4011 from ./raw_images/IMG_7974.JPG
Cropped face 4012 from ./raw_images/IMG_7974.JPG
Cropped face 4013 from ./raw_images/IMG_7974.JPG
Cropped face 4014 from ./raw_images/IMG_7974.JPG
Cropped face 4015 from ./raw_images/IMG_7974.JPG
Cropped face 4016 from ./raw_images/IMG_7974.JPG
Cropped face 4017 from ./raw_images/IMG_7974.JPG
Cropped face 4018 from ./raw_images/IMG_7974.JPG
Cropped face 4019 

/Users/bennyxu/miniforge3/envs/tf_mac/lib/python3.10/site-packages/mtcnn/mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
/Users/bennyxu/miniforge3/envs/tf_mac/lib/python3.10/site-packages/mtcnn/mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
/Users/bennyxu/miniforge3/envs/tf_mac/lib/python3.10/site-packages/mtcnn/mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
/Users/bennyxu/miniforge3/envs/tf_mac/lib/python3.10/site-packages/mtcnn/mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':


KeyboardInterrupt: 